# Experiments

Datasets : [EJUST-GINR1, OU-ISIR, HuGaDB, MMUISD]

**Data Variations**
* Features = [Accelerometer Only, Accelerometer + Gyroscope]
* Sequence Length = [1, 2, 3, 4]
* Overlap = [no overlap, 0.25, 0.5, 0.75]
* Train-Validation-Test split = [0.8, 0.1, 0.1]

**Model Variations**
* Dropout = [Yes, No]
* BN = [Yes, No]
* Weight Decay = [Yes, No]
* Weight Decay + BN
* Weight Decay + Dropout

**Report:**
* Mean EER
* Std EER
* Min EER
* Max EER

**Visualization**
* Distance Vectors of Best and Worst combinations of all datasets stored in 200 ppt.

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!cp -r "/content/drive/My Drive/icmla2020/src" /content/
!mv src/datasets.py datasets.py
!mv src/models.py models.py
!mv src/analyze.py analyze.py
!ls

In [19]:
max_classes = [20, 744, 120]
data_path = ['../data/ejust/', '../data/osaka/', '../data/mmuisd/normal/pocket/']
results_path = '../results/'

In [22]:
import datasets, models, analyze
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
# from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# import nevergrad as ng
np.set_printoptions(precision=4)
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [23]:
def prepare_data(train_sps, val_sps, test_sps, max_cls, split, data, labels, indices_list, run_n):
    train_split = int(split[0] * max_cls)
    val_split = int(split[1] * max_cls)
    test_split = int(split[2] * max_cls)

    rand_indices = np.random.choice(range(max_cls), size=max_cls, replace=False)
    # print('(%d)'%len(rand_indices), rand_indices)
#     indices_list.append(rand_indices)
    print('Run #%d:'%run_n)
    
    all_data_rand, all_labels_rand = [data[x] for x in rand_indices], [labels[x] for x in rand_indices]
    print('all_data_rand:', len(all_data_rand))
    b_data, b_labels = datasets.generate_batch(train_sps, all_data_rand[:train_split], all_labels_rand[:train_split])
    
    val_data, val_labels = datasets.generate_batch(val_sps, all_data_rand[train_split:train_split+val_split], all_labels_rand[train_split:train_split+val_split])
    ridx = np.random.choice(range(b_data[0].shape[0]), size=b_data[0].shape[0], replace=False)
    
    

    l_input = b_data[0][ridx]
    r_input = b_data[1][ridx]
    b_labels = b_labels[ridx]

    # print(l_input[0].shape)

    l_input_val = val_data[0]
    r_input_val = val_data[1]
    if test_split != 0:
        b_data_test, b_labels_test = datasets.generate_batch(test_sps, all_data_rand[train_split+val_split:], all_labels_rand[train_split+val_split:])
        print('test_data length:', len(b_data_test))
        l_input_test = b_data_test[0]
        r_input_test = b_data_test[1]
        return l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test
    else:
        return l_input, r_input, b_labels, l_input_val, r_input_val, val_labels

In [24]:
def load_dataset(path, segLen, overlap, acc_only):
    if path == data_path[0]:
        data, labels = datasets.load_segment_EJUST(path, 'D5-LC', [0, 20], segment_time=segLen)
    elif path == data_path[1]:
        data, labels = datasets.load_segment_osaka(path, 
                                            [0,744], 
                                            sample_rate=100,
                                            acc_only=acc_only,
                                            segment_time=segLen, 
                                            overlapped=overlap, 
                                            overlap=overlap, 
                                            downsample=True)
    elif path == data_path[2]:
        data, labels = datasets.load_segment_mmuisd(path, [0,120], segment_time=segLen)
    else:
        raise ValueError('Incorrect data path is passed.')
    return data, labels

In [25]:
def build_model1(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(64, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [26]:
def build_model2(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
        model.add(Conv1D(32, 3, strides=2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape))
        model.add(Conv1D(32, 3, strides=2, activation='relu'))
    
    if bn:
        model.add(BatchNormalization())

    model.add(MaxPooling1D(pool_size=2))

    if reg:
        model.add(Conv1D(64, 5, strides=2, activation='tanh', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
        model.add(Conv1D(128, 5, strides=3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2, activation='tanh'))
        model.add(Conv1D(128, 5, strides=3, activation='relu'))
    
    if bn:
        model.add(BatchNormalization())

    model.add(GlobalMaxPooling1D())
    return model

In [27]:
def build_model3(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    model.add(Conv1D(64, 5, strides=3, padding='valid', activation='relu', input_shape=input_shape))
    if bn: model.add(BatchNormalization())
    model.add(Conv1D(128, 3, strides=2, padding='valid', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, 2, strides=1, padding='valid', activation='tanh'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(GlobalMaxPooling1D())
    return model

In [28]:
def build_model4(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(128, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(128, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [29]:
def build_model5(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(256, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(256, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [30]:
# acc_only_list    = [False, True]
# segLen_list      = [1, 2]
# overlap_list     = [0, 0.25, 0.5, 0.75]
# # models_list      = [build_model1, build_model2, build_model3]
# models_list      = [build_model4, build_model5]
# bn_list          = [True, False]
# reg_list         = [True, False]

# Configurations
configs = [# EJUST
    {'train_sps': 500,
     'val_sps': 200,
     'test_sps': 200,
     'max_cls': 20,
     'split': [0.75, 0.25, 0],
     'runs': 10,
     'overlap': 0 
    },
    { # OSAKA
        'train_sps': 50,
        'val_sps': 20,
        'test_sps': 20,
        'max_cls': 740,
        'split': [0.9, 0.1, 0],
        'runs': 10,
        'overlap': 0.75
    },
    { # MMUISD
        'train_sps': 500,
        'val_sps': 200,
        'test_sps': 200,
        'max_cls': 120,
        'split': [0.9, 0.1, 0],
        'runs': 10,
        'overlap': 0
    },
    
]
# train_sps       = 500
# val_sps         = 200
# test_sps        = 200
# max_cls         = 20
# split           = [0.5, 0.25, 0.25]
output_dropout  = 0.1
# runs            = 30
np.set_printoptions(precision=4)

Here I want to do Transfer Learning which means, training on one dataset, taking the trained model and apply it on other datasets, re-training the final FCL only. Problem is, which model you should choose? Maybe train multiple models and take the best one; however, this may be a model trained on good combination only. Well, this is fine, you can train it on the best combination however, the results on other datasets may not be informative; well, you will train on the whole dataset and leave only 0.1 for validation. Then use the model on the whole data (there is no combination here almost) to figure out the performance on other datasets. The whole setup of the previous experiments may not work well here.

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Model
from tensorflow.keras.layers import Input

# training
datasets_indices = {0,1,2}
# datasets_indices = {1,2,3}
TL_mat = np.zeros((3,3,10))
datasets_list = []

for i in datasets_indices:
    print('loading', data_path[i])
    datasets_list.append(load_dataset(data_path[i], 2, configs[i]['overlap'], acc_only=False))

for i in datasets_indices:
#     i = 2
    print('Train on', data_path[i])
    for run_n in range(configs[i]['runs']):
        data, labels = datasets_list[i]
        print('data length:', len(data))
        print('data loaded')
        l_input, r_input, b_labels, l_input_val, r_input_val, val_labels = prepare_data(configs[i]['train_sps'],
                                                                                        configs[i]['val_sps'],
                                                                                        configs[i]['test_sps'],
                                                                                        configs[i]['max_cls'],
                                                                                        configs[i]['split'],
                                                                                        data,
                                                                                        labels,
                                                                                        [],
                                                                                        run_n)
        print('pairs created')
        adam = Adam(learning_rate=0.001)
        earlystop = EarlyStopping(monitor='val_accuracy', patience=5)
        
        bn = 1
        reg = 0
        print('hyperparameteres specified')
        cnn = build_model5(l_input[0].shape, bn, reg)
        print('CNN loaded')
        siamese = models.build_siamese(l_input[0].shape, cnn, output_dropout)
        print('Siamese loaded')
        siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
        print('model compiled')
        if run_n == 0: 
            cnn.summary()
        hist = siamese.fit([l_input, r_input], 
                            b_labels, 
                            shuffle=True,
                            batch_size=64,
                            epochs=100,
                            callbacks=[earlystop],
                            validation_data=([l_input_val, r_input_val], val_labels),
                            verbose=2
                            )

        # testing
        for j in datasets_indices - {i}:
            print('Test on', data_path[j])
            data, labels = datasets_list[j]
            l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test = prepare_data(configs[j]['train_sps'], \
                                                                                        configs[j]['val_sps'], \
                                                                                        configs[j]['test_sps'], \
                                                                                        configs[j]['max_cls'], \
                                                                                        [0.2, 0.2, 0.6], \
                                                                                        data[:], \
                                                                                        labels[:], \
                                                                                        [], \
                                                                                        run_n)
            
            feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], 
                                       outputs=siamese.get_layer('lambda').output)
            d_vect = feature_exctractor.predict([l_input, r_input])
            d_vect_val = feature_exctractor.predict([l_input_val, r_input_val])
            d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])

            FCL = Sequential()
#             FCL.add(Input(d_vect.shape[1:]))
            FCL.add(Dropout(rate=0.1))
            FCL.add(Dense(1, activation='sigmoid', input_shape=d_vect.shape[1:]))
            
            FCL.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
            hist = FCL.fit(d_vect, \
                            b_labels, \
                            shuffle=True, \
                            batch_size=64, \
                            epochs=20, \
                            callbacks=[earlystop], \
                            validation_data=(d_vect_val, val_labels), \
                            verbose=2
                            )
            FRR, FAR, EER, EER_th = analyze.ROC(FCL, d_vect_test, b_labels_test)
            print('EER:', EER)
            TL_mat[i,j, run_n] = EER
            np.save(results_path + 'TL_mat_again.npy', TL_mat)
            del data, labels, l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test
#             break

loading ../data/ejust/
loading ../data/osaka/
loading ../data/mmuisd/normal/pocket/
Train on ../data/ejust/
data length: 20
data loaded
Run #0:
all_data_rand: 20
pairs created
hyperparameteres specified
CNN loaded
Siamese loaded
model compiled
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 98, 64)            1216      
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 64)            256       
_________________________________________________________________
activation (Activation)      (None, 98, 64)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 23, 256)           82176     
__________

Epoch 4/100
235/235 - 2s - loss: 0.0249 - accuracy: 0.9956 - val_loss: 0.1716 - val_accuracy: 0.9335
Epoch 5/100
235/235 - 2s - loss: 0.0175 - accuracy: 0.9970 - val_loss: 0.1567 - val_accuracy: 0.9400
Epoch 6/100
235/235 - 2s - loss: 0.0130 - accuracy: 0.9975 - val_loss: 0.2461 - val_accuracy: 0.9150
Epoch 7/100
235/235 - 2s - loss: 0.0196 - accuracy: 0.9963 - val_loss: 0.2060 - val_accuracy: 0.9240
Epoch 8/100
235/235 - 2s - loss: 0.0081 - accuracy: 0.9992 - val_loss: 0.1805 - val_accuracy: 0.9425
Epoch 9/100
235/235 - 2s - loss: 0.0085 - accuracy: 0.9985 - val_loss: 0.1758 - val_accuracy: 0.9415
Epoch 10/100
235/235 - 2s - loss: 0.0123 - accuracy: 0.9982 - val_loss: 0.2103 - val_accuracy: 0.9320
Epoch 11/100
235/235 - 2s - loss: 0.0079 - accuracy: 0.9987 - val_loss: 0.1644 - val_accuracy: 0.9410
Epoch 12/100
235/235 - 2s - loss: 0.0070 - accuracy: 0.9988 - val_loss: 0.1787 - val_accuracy: 0.9435
Epoch 13/100
235/235 - 2s - loss: 0.0063 - accuracy: 0.9991 - val_loss: 0.1851 - val_acc

EER: 0.12645833333333334
data length: 20
data loaded
Run #4:
all_data_rand: 20
pairs created
hyperparameteres specified
CNN loaded
Siamese loaded
model compiled
Epoch 1/100
235/235 - 4s - loss: 0.1865 - accuracy: 0.9464 - val_loss: 0.2374 - val_accuracy: 0.9290
Epoch 2/100
235/235 - 2s - loss: 0.0567 - accuracy: 0.9899 - val_loss: 0.2729 - val_accuracy: 0.9235
Epoch 3/100
235/235 - 2s - loss: 0.0320 - accuracy: 0.9945 - val_loss: 0.3211 - val_accuracy: 0.9190
Epoch 4/100
235/235 - 2s - loss: 0.0201 - accuracy: 0.9967 - val_loss: 0.2744 - val_accuracy: 0.9290
Epoch 5/100
235/235 - 2s - loss: 0.0121 - accuracy: 0.9990 - val_loss: 0.2620 - val_accuracy: 0.9295
Epoch 6/100
235/235 - 2s - loss: 0.0101 - accuracy: 0.9987 - val_loss: 0.3362 - val_accuracy: 0.9205
Epoch 7/100
235/235 - 2s - loss: 0.0066 - accuracy: 0.9995 - val_loss: 0.3978 - val_accuracy: 0.9200
Epoch 8/100
235/235 - 2s - loss: 0.0056 - accuracy: 0.9992 - val_loss: 0.3410 - val_accuracy: 0.9265
Epoch 9/100
235/235 - 2s - loss

Epoch 12/20
375/375 - 1s - loss: 0.2123 - accuracy: 0.9222 - val_loss: 0.2458 - val_accuracy: 0.9107
Epoch 13/20
375/375 - 1s - loss: 0.2128 - accuracy: 0.9207 - val_loss: 0.2493 - val_accuracy: 0.9064
Epoch 14/20
375/375 - 1s - loss: 0.2091 - accuracy: 0.9202 - val_loss: 0.2473 - val_accuracy: 0.9078
Epoch 15/20
375/375 - 1s - loss: 0.2089 - accuracy: 0.9226 - val_loss: 0.2527 - val_accuracy: 0.9042
Epoch 16/20
375/375 - 1s - loss: 0.2078 - accuracy: 0.9227 - val_loss: 0.2711 - val_accuracy: 0.8941
Epoch 17/20
375/375 - 1s - loss: 0.2053 - accuracy: 0.9236 - val_loss: 0.2584 - val_accuracy: 0.8985
EER: 0.1398611111111111
data length: 20
data loaded
Run #6:
all_data_rand: 20
pairs created
hyperparameteres specified
CNN loaded
Siamese loaded
model compiled
Epoch 1/100
235/235 - 4s - loss: 0.1861 - accuracy: 0.9475 - val_loss: 0.3417 - val_accuracy: 0.8870
Epoch 2/100
235/235 - 2s - loss: 0.0561 - accuracy: 0.9891 - val_loss: 0.3551 - val_accuracy: 0.8915
Epoch 3/100
235/235 - 2s - loss:

Epoch 4/20
232/232 - 1s - loss: 0.5438 - accuracy: 0.7803 - val_loss: 0.5632 - val_accuracy: 0.7578
Epoch 5/20
232/232 - 1s - loss: 0.5326 - accuracy: 0.7784 - val_loss: 0.5561 - val_accuracy: 0.7546
Epoch 6/20
232/232 - 1s - loss: 0.5241 - accuracy: 0.7776 - val_loss: 0.5508 - val_accuracy: 0.7546
Epoch 7/20
232/232 - 1s - loss: 0.5162 - accuracy: 0.7773 - val_loss: 0.5463 - val_accuracy: 0.7535
EER: 0.23164414414414414
Test on ../data/mmuisd/normal/pocket/
Run #7:
all_data_rand: 120
test_data length: 2
Epoch 1/20
375/375 - 1s - loss: 0.3031 - accuracy: 0.9014 - val_loss: 0.3194 - val_accuracy: 0.8808
Epoch 2/20
375/375 - 1s - loss: 0.2232 - accuracy: 0.9346 - val_loss: 0.3078 - val_accuracy: 0.8758
Epoch 3/20
375/375 - 1s - loss: 0.2004 - accuracy: 0.9392 - val_loss: 0.3006 - val_accuracy: 0.8752
Epoch 4/20
375/375 - 1s - loss: 0.1895 - accuracy: 0.9424 - val_loss: 0.3002 - val_accuracy: 0.8757
Epoch 5/20
375/375 - 1s - loss: 0.1807 - accuracy: 0.9428 - val_loss: 0.2974 - val_accurac

...

Epoch 13/100
1688/1688 - 15s - loss: 0.0106 - accuracy: 0.9970 - val_loss: 0.2159 - val_accuracy: 0.9544
Epoch 14/100
1688/1688 - 13s - loss: 0.0100 - accuracy: 0.9972 - val_loss: 0.1864 - val_accuracy: 0.9458
Epoch 15/100
1688/1688 - 13s - loss: 0.0102 - accuracy: 0.9970 - val_loss: 0.2502 - val_accuracy: 0.9421
Epoch 16/100
1688/1688 - 13s - loss: 0.0083 - accuracy: 0.9976 - val_loss: 0.2710 - val_accuracy: 0.9446
Epoch 17/100
1688/1688 - 13s - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.1965 - val_accuracy: 0.9477
Epoch 18/100
1688/1688 - 13s - loss: 0.0074 - accuracy: 0.9978 - val_loss: 0.3182 - val_accuracy: 0.9365
Test on ../data/ejust/
Run #3:
all_data_rand: 20
test_data length: 2
Epoch 1/20
63/63 - 1s - loss: 0.5748 - accuracy: 0.7218 - val_loss: 0.3419 - val_accuracy: 0.8587
Epoch 2/20
63/63 - 0s - loss: 0.2766 - accuracy: 0.9095 - val_loss: 0.2816 - val_accuracy: 0.8988
Epoch 3/20
63/63 - 0s - loss: 0.2123 - accuracy: 0.9395 - val_loss: 0.2689 - val_accuracy: 0.9025
Epoch 4

EER: 0.15788288288288288
data length: 120
data loaded
Run #5:
all_data_rand: 120
pairs created
hyperparameteres specified
CNN loaded
Siamese loaded
model compiled
Epoch 1/100
1688/1688 - 17s - loss: 0.1408 - accuracy: 0.9566 - val_loss: 0.3021 - val_accuracy: 0.8915
Epoch 2/100
1688/1688 - 14s - loss: 0.0663 - accuracy: 0.9799 - val_loss: 0.2304 - val_accuracy: 0.9233
Epoch 3/100
1688/1688 - 14s - loss: 0.0468 - accuracy: 0.9855 - val_loss: 0.2778 - val_accuracy: 0.9275
Epoch 4/100
1688/1688 - 15s - loss: 0.0355 - accuracy: 0.9892 - val_loss: 0.2135 - val_accuracy: 0.9371
Epoch 5/100
1688/1688 - 14s - loss: 0.0274 - accuracy: 0.9920 - val_loss: 0.2044 - val_accuracy: 0.9381
Epoch 6/100
1688/1688 - 15s - loss: 0.0230 - accuracy: 0.9932 - val_loss: 0.2314 - val_accuracy: 0.9440
Epoch 7/100
1688/1688 - 15s - loss: 0.0204 - accuracy: 0.9941 - val_loss: 0.2752 - val_accuracy: 0.9404
Epoch 8/100
1688/1688 - 15s - loss: 0.0191 - accuracy: 0.9942 - val_loss: 0.2005 - val_accuracy: 0.9415
Epoch

Epoch 6/20
232/232 - 1s - loss: 0.4680 - accuracy: 0.8169 - val_loss: 0.4676 - val_accuracy: 0.8117
Epoch 7/20
232/232 - 1s - loss: 0.4565 - accuracy: 0.8197 - val_loss: 0.4596 - val_accuracy: 0.8122
Epoch 8/20
232/232 - 1s - loss: 0.4497 - accuracy: 0.8237 - val_loss: 0.4530 - val_accuracy: 0.8125
Epoch 9/20
232/232 - 1s - loss: 0.4424 - accuracy: 0.8241 - val_loss: 0.4471 - val_accuracy: 0.8162
Epoch 10/20
232/232 - 1s - loss: 0.4375 - accuracy: 0.8270 - val_loss: 0.4412 - val_accuracy: 0.8152
Epoch 11/20
232/232 - 1s - loss: 0.4315 - accuracy: 0.8259 - val_loss: 0.4366 - val_accuracy: 0.8174
Epoch 12/20
232/232 - 1s - loss: 0.4262 - accuracy: 0.8290 - val_loss: 0.4325 - val_accuracy: 0.8199
Epoch 13/20
232/232 - 1s - loss: 0.4215 - accuracy: 0.8287 - val_loss: 0.4282 - val_accuracy: 0.8191
Epoch 14/20
232/232 - 1s - loss: 0.4176 - accuracy: 0.8324 - val_loss: 0.4245 - val_accuracy: 0.8218
Epoch 15/20
232/232 - 1s - loss: 0.4136 - accuracy: 0.8349 - val_loss: 0.4211 - val_accuracy: 0

Epoch 13/100
1688/1688 - 13s - loss: 0.0113 - accuracy: 0.9969 - val_loss: 0.3218 - val_accuracy: 0.9190
Epoch 14/100
1688/1688 - 12s - loss: 0.0104 - accuracy: 0.9969 - val_loss: 0.2303 - val_accuracy: 0.9483
Epoch 15/100
1688/1688 - 12s - loss: 0.0083 - accuracy: 0.9977 - val_loss: 0.2329 - val_accuracy: 0.9460
Epoch 16/100
1688/1688 - 13s - loss: 0.0099 - accuracy: 0.9970 - val_loss: 0.3745 - val_accuracy: 0.9162
Epoch 17/100
1688/1688 - 13s - loss: 0.0077 - accuracy: 0.9977 - val_loss: 0.2624 - val_accuracy: 0.9398
Epoch 18/100
1688/1688 - 12s - loss: 0.0075 - accuracy: 0.9976 - val_loss: 0.2710 - val_accuracy: 0.9300
Epoch 19/100
1688/1688 - 14s - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.3237 - val_accuracy: 0.9262
Test on ../data/ejust/
Run #8:
all_data_rand: 20
test_data length: 2
Epoch 1/20
63/63 - 1s - loss: 0.5745 - accuracy: 0.7032 - val_loss: 0.5240 - val_accuracy: 0.7169
Epoch 2/20
63/63 - 0s - loss: 0.3006 - accuracy: 0.9075 - val_loss: 0.5763 - val_accuracy: 0.7138


Epoch 11/20
232/232 - 1s - loss: 0.4093 - accuracy: 0.8303 - val_loss: 0.4618 - val_accuracy: 0.7990
Epoch 12/20
232/232 - 1s - loss: 0.4053 - accuracy: 0.8364 - val_loss: 0.4579 - val_accuracy: 0.7998
Epoch 13/20
232/232 - 1s - loss: 0.4018 - accuracy: 0.8341 - val_loss: 0.4550 - val_accuracy: 0.7951
Epoch 14/20
232/232 - 1s - loss: 0.3973 - accuracy: 0.8354 - val_loss: 0.4512 - val_accuracy: 0.8002
Epoch 15/20
232/232 - 1s - loss: 0.3929 - accuracy: 0.8374 - val_loss: 0.4478 - val_accuracy: 0.8020
Epoch 16/20
232/232 - 1s - loss: 0.3915 - accuracy: 0.8360 - val_loss: 0.4456 - val_accuracy: 0.8017
Epoch 17/20
232/232 - 1s - loss: 0.3861 - accuracy: 0.8400 - val_loss: 0.4425 - val_accuracy: 0.8035
Epoch 18/20
232/232 - 1s - loss: 0.3847 - accuracy: 0.8376 - val_loss: 0.4405 - val_accuracy: 0.8051
Epoch 19/20
232/232 - 1s - loss: 0.3806 - accuracy: 0.8414 - val_loss: 0.4380 - val_accuracy: 0.8076
Epoch 20/20
232/232 - 1s - loss: 0.3754 - accuracy: 0.8424 - val_loss: 0.4356 - val_accurac

In [12]:
# data, labels = load_dataset(2, 0, acc_only=False)

# # Results of this configuration/experiment
# EERs = np.array([np.zeros(runs) for _ in range(3*2*2)])

# for run_n in range(runs):
#     l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test = prepare_data(train_sps,
#                                                                                                                                 val_sps,
#                                                                                                                                 test_sps,
#                                                                                                                                 max_cls,
#                                                                                                                                 split,
#                                                                                                                                 data,
#                                                                                                                                 labels,
#                                                                                                                                 [],
#                                                                                                                                 run_n)
#     adam = Adam(learning_rate=0.001)
#     earlystop = EarlyStopping(monitor='val_accuracy', patience=5)
#     for bn in bn_list:
#         for reg in reg_list:
#             for model_index, build_cnn in enumerate(models_list):
#                 cnn = build_cnn(l_input[0].shape, bn, reg)
#                 siamese = models.build_siamese(l_input[0].shape, cnn, output_dropout)
#                 siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
#                 if run_n == 0: 
#                     cnn.summary()
#                 hist = siamese.fit([l_input, r_input], 
#                                     b_labels, 
#                                     shuffle=True,
#                                     batch_size=64,
#                                     epochs=100,
#                                     callbacks=[earlystop],
#                                     validation_data=([l_input_val, r_input_val], val_labels),
#                                     verbose=2
#                                     )
#                 FRR, FAR, EER, EER_th = analyze.ROC(siamese, [l_input_test, r_input_test], b_labels_test)
#                 print('MODEL(%d) BN(%d) REG(%d) EER: %g', (model_index, bn, reg, EER))
#                 EERs[4 * model_index + 2*bn + reg , run_n] = EER
#                 np.save(results_path + 'ejust_filters_eers.npy', EERs)


In [13]:
# EERs = np.load(results_path + 'ejust_filters_eers.npy')
# EERs = EERs[:8]
# print(EERs.shape)

In [14]:
# # EERs[EERs == 0] = 100
# means = np.mean(EERs, axis=1) * 100
# print(means)

# stds = np.std(EERs, axis=1) * 100
# print(stds)

# mins = np.min(EERs, axis=1) * 100
# print(mins)

# mins = np.max(EERs, axis=1) * 100
# print(mins)

In [15]:
# import pandas as pd
# E = pd.DataFrame(EERs)
# print(E.iloc[:,:15])